### Запуск PySpark

In [1]:
import findspark
import re
import ast
import string
findspark.init()

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="practise")

In [3]:
letters = list(string.ascii_letters)
letters[:10]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [4]:
letters_rdd = sc.parallelize(letters)
letters_rdd.getNumPartitions() 

8

In [5]:
letters_rdd.glom().collect()

[['a', 'b', 'c', 'd', 'e', 'f'],
 ['g', 'h', 'i', 'j', 'k', 'l'],
 ['m', 'n', 'o', 'p', 'q', 'r'],
 ['s', 't', 'u', 'v', 'w', 'x'],
 ['y', 'z', 'A', 'B', 'C', 'D'],
 ['E', 'F', 'G', 'H', 'I', 'J'],
 ['K', 'L', 'M', 'N', 'O', 'P'],
 ['Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']]

In [6]:
letters_rdd = (
    letters_rdd
    .map(lambda x: (x, 1))
)

In [7]:
letters_rdd.first()

('a', 1)

### Задачи на Spark RDD и PairRDD

## Задача 1. Разминка

Вам нужно провести аналитику с помощью Spark на основе датасета о марках машин.
Ваша задача - посчитать, сколько марок машин есть в датасете по интересующим
срезам (например - по первой букве в названии марки).

Получите RDD, содержащий количество производителей автомобилей с разбиением
по буквам, с которых начинается название производителя (буквы в разном регистре
будем считать разными). Если для какой-то буквы нет ни одного производителя,
количество должно быть равно 0.

In [8]:
# !wget https://raw.githubusercontent.com/big-data-team/big-data-course/master/public_examples/spark/rdd/workshop/car_brands.txt

In [9]:
cars_rdd = sc.textFile("car_brands.txt")
cars_rdd.count()

64

In [10]:
cars_count = (
    cars_rdd
    .map(lambda x: (x[0], 1))
    .reduceByKey(lambda x, y: x+y)
)
cars_count.first()

('C', 4)

In [11]:
joined_rdd = letters_rdd.leftOuterJoin(cars_count)
joined_rdd.first()

('O', (1, 1))

In [12]:
result_rdd = (
    joined_rdd
    .map(lambda y: (y[0], 0) if not y[1][1] else (y[0], y[1][1]))
)

In [13]:
result_rdd.collect()

[('O', 1),
 ('m', 0),
 ('u', 0),
 ('A', 4),
 ('H', 3),
 ('U', 0),
 ('V', 2),
 ('y', 0),
 ('R', 3),
 ('S', 7),
 ('e', 0),
 ('f', 0),
 ('k', 0),
 ('n', 0),
 ('o', 0),
 ('t', 0),
 ('I', 2),
 ('b', 0),
 ('h', 0),
 ('i', 0),
 ('p', 0),
 ('r', 0),
 ('s', 0),
 ('J', 2),
 ('K', 2),
 ('W', 0),
 ('q', 0),
 ('x', 0),
 ('F', 4),
 ('Q', 0),
 ('T', 2),
 ('X', 0),
 ('Z', 0),
 ('c', 0),
 ('j', 0),
 ('l', 0),
 ('N', 1),
 ('B', 3),
 ('D', 6),
 ('G', 0),
 ('P', 2),
 ('d', 0),
 ('g', 0),
 ('C', 4),
 ('L', 7),
 ('a', 0),
 ('v', 0),
 ('w', 0),
 ('z', 0),
 ('E', 0),
 ('M', 9),
 ('Y', 0)]

In [14]:
#!wget https://raw.githubusercontent.com/big-data-team/big-data-course/master/public_examples/spark/rdd/workshop/cities.jsonlines

### Задача 2. Конкурсы и шарады с костылями и приседаниями

В этом задании вам предстоит поработать с “грязными” данными (все как в реальной
жизни). Возьмите битый датасет, который содержит статистику о разных городах. В каждой строке находится строка в формате json, вам нужно построить на основе
этих данных RDD из словарей с условием, что:

 - если континент отсутствует - ставим Earth;
 - если отсутствует population - ставим 0;
 - если это невалидная запись json, то такую запись игнорируем;
 
После этого можем переходить непосредственно к аналитике на Spark:
1. Найдите все дубликаты и создайте новый RDD без дублей;
2. количество городов и общая сумма населения;
3. Найдите наиболее населенный город

In [15]:
rows_json = sc.textFile("cities.jsonlines")
rows_json.collect()

['{ "name":"Moscow", "country":"Russia", "continent": "Europe", "population": 12380664}',
 '{ "name":"Madrid", "country":"Spain" }',
 '{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 2196936}',
 '{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 3490105}',
 '{ "name":"Barselona", "country":"Spain", "continent": "Europe" }',
 '{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }',
 '{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }',
 '{ "name":"New York, "country":"USA"']

In [16]:
def to_dict(x):
    try:
        r = ast.literal_eval(x)
        return r
    except:
        pass

In [18]:
cities = (
        rows_json
        .map(to_dict)
        .filter(lambda x: x is not None)
)
cities.collect()

[{'name': 'Moscow',
  'country': 'Russia',
  'continent': 'Europe',
  'population': 12380664},
 {'name': 'Madrid', 'country': 'Spain'},
 {'name': 'Paris',
  'country': 'France',
  'continent': 'Europe',
  'population': 2196936},
 {'name': 'Berlin',
  'country': 'Germany',
  'continent': 'Europe',
  'population': 3490105},
 {'name': 'Barselona', 'country': 'Spain', 'continent': 'Europe'},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948}]

In [19]:
def correct_dict(x):
    if "continent" not in x.keys():
        x['continent'] = "Earth"
    if 'population' not in x.keys():
        x['population'] = 0
    return x

In [20]:
cities = (
    cities
    .map(correct_dict)
)
cities.collect()

[{'name': 'Moscow',
  'country': 'Russia',
  'continent': 'Europe',
  'population': 12380664},
 {'name': 'Madrid', 'country': 'Spain', 'continent': 'Earth', 'population': 0},
 {'name': 'Paris',
  'country': 'France',
  'continent': 'Europe',
  'population': 2196936},
 {'name': 'Berlin',
  'country': 'Germany',
  'continent': 'Europe',
  'population': 3490105},
 {'name': 'Barselona',
  'country': 'Spain',
  'continent': 'Europe',
  'population': 0},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948}]

In [21]:
def get_tuple(x):
    if "continent" not in x.keys():
        x['continet'] = "Earth"
    if 'population' not in x.keys():
        x['population'] = 0
    return (x['name'], x['country'], x['continent'], x['population'])

In [22]:
cities = (
    cities
    .map(get_tuple)
)

In [23]:
cities.collect()

[('Moscow', 'Russia', 'Europe', 12380664),
 ('Madrid', 'Spain', 'Earth', 0),
 ('Paris', 'France', 'Europe', 2196936),
 ('Berlin', 'Germany', 'Europe', 3490105),
 ('Barselona', 'Spain', 'Europe', 0),
 ('Cairo', 'Egypt', 'Africa', 11922948),
 ('Cairo', 'Egypt', 'Africa', 11922948)]

In [24]:
cities = cities.distinct()
cities.collect()

[('Paris', 'France', 'Europe', 2196936),
 ('Berlin', 'Germany', 'Europe', 3490105),
 ('Barselona', 'Spain', 'Europe', 0),
 ('Moscow', 'Russia', 'Europe', 12380664),
 ('Madrid', 'Spain', 'Earth', 0),
 ('Cairo', 'Egypt', 'Africa', 11922948)]

In [25]:
cities.getNumPartitions()

2

In [26]:
# количество городов и сумма населения;
seqOp = (lambda local_result, list_element: (local_result[0] + list_element[-1], local_result[1] + 1) )
combOp = (lambda some_local_result, another_local_result: (some_local_result[0] + another_local_result[0], some_local_result[1] + another_local_result[1]) )
cities.aggregate((0, 0), seqOp, combOp)

(29990653, 6)

In [27]:
# Найдите наиболее населенный город;
cities.max(lambda x: x[-1])[0]

'Moscow'